## Imports


In [1]:
import distributed
import pooch
import s3fs
import xarray as xr

In [ ]:
client = distributed.Client()
client

## Shared 

In [4]:
zenodo_zipped_url = 'https://zenodo.org/records/8293998/files/code_and_data.zip'
access_key_id = '<ADD>'
secret_access_key = '<ADD>'
endpoint_url = 'https://nyu1.osn.mghpcc.org'
write_fs = s3fs.S3FileSystem(
    key=access_key_id, secret=secret_access_key, client_kwargs={'endpoint_url': endpoint_url}
)

## Raw Training Data

In [ ]:
training_data_raw_path = 'code_and_data/Data/training_data_raw_time_index_49_149.nc'

raw_processor = pooch.Unzip(members=[training_data_raw_path])
raw_file = pooch.retrieve(url=zenodo_zipped_url, known_hash=None, processor=raw_processor)

raw_ds = xr.open_dataset(raw_file[0])
raw_ds = raw_ds.chunk({'indexes': 2500000, 'sigma': 5})
raw_ds.to_zarr(
    write_fs.get_mapper('leap-pangeo-manual/GOTM/raw_training_data.zarr'),
    mode='w',
    consolidated=False,
)

## Training Data SF

In [ ]:
training_data_SF_path = 'code_and_data/Data/training_data_for_SF_hbl_gaps_filled.nc'

sf_processor = pooch.Unzip(members=[training_data_SF_path])
sf_file = pooch.retrieve(url=zenodo_zipped_url, known_hash=None, processor=sf_processor)

sf_ds = xr.open_dataset(sf_file[0], engine='netcdf4')
sf_ds = sf_ds.chunk({'indexes': 2500000, 'sigma': 5})
sf_ds.to_zarr(write_fs.get_mapper('leap-pangeo-manual/GOTM/sf_training_data.zarr'), mode='w')